In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score ,recall_score,f1_score
from tqdm import tqdm


In [2]:
#List of  Genres learnt from the training data
genre_list =['action','adult','adventure','animation','biography','comedy','crime','documentary','family','fantasy','game_show','history','horror','music','musical','mystery','news','reality_tv','romance','sci_fi','short','sport','talk_shoe','thriller','war','western']



In [3]:
#define a fallback genre for movies which the model findds very hard to predict
fallback_genre='Unknown'

In [4]:
#Load the Traning dataset from train_data.txt
try:
    with tqdm(total=50,desc="Loading Train Data") as pbar:
        train_data = pd.read_csv('train_data.txt', sep=':::', header=None, names=['SerialNumber','MOVIE_NAME','GENRE','MOVIE_PLOT'],engine='python')                               
        pbar.update(50)
except Exception as e:
    print(f"Error loading train_data: {e}")
    raise

Loading Train Data: 100%|█████████████████| 50/50 [00:00<00:00, 173.66it/s]


In [5]:
#Data preprocessing for the training data
x_train = train_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())
genre_labels = [genre.split(',') for genre in train_data['GENRE']]
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(genre_labels)

In [6]:
#TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000) #you can adjust max_features



In [7]:
#Fit and transform the training data with progress bar
with tqdm(total=50,desc="Vectorizing Training Data") as pbar:
    x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
    pbar.update(50)


Vectorizing Training Data: 100%|███████████| 50/50 [00:03<00:00, 13.91it/s]


In [8]:
#Train a multiOutput Naive Bayes classifier using the training data    
with tqdm(total=50,desc="Training Model") as pbar:
    naive_bayes = MultinomialNB()
    multi_output_classifier = MultiOutputClassifier(naive_bayes)
    multi_output_classifier.fit(x_train_tfidf,y_train)
    pbar.update(50)

Training Model: 100%|██████████████████████| 50/50 [00:00<00:00, 58.49it/s]


In [9]:
#Load your test dataset from test_data.txt
try:
    with tqdm(total=50, desc="Loading test data")as pbar:
        test_data = pd.read_csv('test_data.txt', sep=':::', header=None, names=['SerialNumber','MOVIE_NAME','MOVIE_PLOT',],engine='python')
        pbar.update(50)
except Exception as e:
    print(f"Error loading test_data: {e}")
    raise

Loading test data: 100%|██████████████████| 50/50 [00:00<00:00, 105.20it/s]


In [10]:
#Data preprocessing for test data
x_test = test_data['MOVIE_PLOT'].astype(str).apply(lambda doc: doc.lower())

In [11]:
# Transform the test data with progress bar
with tqdm(total=50, desc="Vectorizing Test Data") as pbar:
    x_test_tfidf = tfidf_vectorizer.transform(x_test)
    pbar.update(50)

Vectorizing Test Data: 100%|███████████████| 50/50 [00:06<00:00,  7.49it/s]


In [12]:
# predict genres on the test data
with tqdm(total=50,desc="Predicting on Test Data") as pbar:
    y_pred = multi_output_classifier.predict(x_test_tfidf)
    pbar.update(50)
                                    

Predicting on Test Data: 100%|█████████████| 50/50 [00:00<00:00, 84.42it/s]


In [13]:
#Create a DataFrame for test data with movie names and predicted genres
test_movie_names = test_data['MOVIE_NAME']
predicted_genres = mlb.inverse_transform(y_pred)
test_results = pd.DataFrame({'MOVIE_NAME': test_movie_names, 'PREDICTED_GENRES': predicted_genres})

In [14]:
# Replace empty unpredicted genres with the fallback genre 
test_results['PREDICTED_GENRES'] = test_results['PREDICTED_GENRES'].apply(lambda genres: [fallback_genre] if len(genres) ==0 else genres)


In [15]:
# Write the results to an output text file with proper formatting
with open("model_evaluation.txt","w", encoding="utf-8")as output_file:
    for _, row in test_results.iterrows():
        movie_name = row['MOVIE_NAME']
        genre_str = ', '.join(row['PREDICTED_GENRES'])
        output_file.write(f"{movie_name} ::: {genre_str}\n")

In [16]:
# Calculate evaluation metrics using training labels (as a proxy)
y_train_pred = multi_output_classifier.predict(x_train_tfidf)

In [17]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_train, y_train_pred)
precision = precision_score(y_train,y_train_pred, average='micro')
recall = recall_score(y_train,y_train_pred, average='micro')
f1 = f1_score(y_train, y_train_pred, average='micro')

In [18]:
# Append the evaluation metrics to the output file
with open("model_evaluation.txt", "a", encoding="utf-8") as output_file:
    output_file.write("\n\nModel Evalution Metrics:\n")
    output_file.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    output_file.write(f"Precision: {precision:.2f}\n")
    output_file.write(f"Recall: {recall:.2f}\n")
    output_file.write(f"F1-score: {f1:.2f}\n")

    

In [19]:
print("Model evaluation results and metrics have been saved to 'model_evaluation.txt'.")

Model evaluation results and metrics have been saved to 'model_evaluation.txt'.
